# Bosonic Dimer

In [ ]:
using Pkg; Pkg.activate()
using KadanoffBaym
using LinearAlgebra

using FFTW, Interpolations

In [ ]:
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")
using LaTeXStrings

## Model

### Non-Hermitian Hamiltonian

$$
    \hat{H} = \left(\omega_1 -i\frac{\lambda}{2}\right) a^{\dagger}_1 a^\phantom{\dagger}_1 -i\frac{\gamma}{2} a^\phantom{\dagger}_1 a^{\dagger}_1  + \omega_2 a^{\dagger}_2 a^\phantom{\dagger}_2 + J \left(a^{\dagger}_1 a^\phantom{\dagger}_2 + a^{\dagger}_2 a^\phantom{\dagger}_1\right)
$$

### Master equation

$$
    \partial_{t} \hat{\rho}=-i\left[\hat{H} \hat{\rho}-\hat{\rho} \hat{H}^{
    \dagger}\right]+\lambda a^\phantom{\dagger}_1 \hat{\rho}  a^{\dagger}_1 +\gamma  a^{\dagger}_1 \hat{\rho} a^\phantom{\dagger}_1
$$

### Equations of motion

#### Vertical Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    i \partial_t - \omega_1 + i(\lambda + \gamma)/2 & -J \\
    -J & i \partial_t - \omega_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t') 
    - i\gamma
    \begin{pmatrix}
    G^{\tilde{T}}_{11} & G^{\tilde{T}}_{12} \\
    0 & 0
    \end{pmatrix}(t, t')  \\
    0 &= \begin{pmatrix}
    i \partial_t - \omega_1 - i(\lambda + \gamma)/2 & -J \\
    -J & i \partial_t - \omega_2
    \end{pmatrix} 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t')  
    + i\lambda
    \begin{pmatrix}
    G^T_{11} & G^T_{12} \\
    0 & 0
    \end{pmatrix}(t, t')   
\end{split}\end{align}

#### Horizontal Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \omega_1 + i(\lambda + \gamma)/2 & J \\
    J & i \partial_{t'} + \omega_2
    \end{pmatrix} 
    - i\gamma
    \begin{pmatrix}
    G^{{T}}_{11} & 0 \\
    G^{{T}}_{21} & 0
    \end{pmatrix}(t, t')     \\
    0 &= \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(t, t') 
    \begin{pmatrix}
    i \partial_{t'} + \omega_1 - i(\lambda + \gamma)/2 & J \\
    J & i \partial_{t'} + \omega_2
    \end{pmatrix}  
    + i\lambda
    \begin{pmatrix}
    G^{\tilde{T}}_{11} & 0 \\
    G^{\tilde{T}}_{21} & 0
    \end{pmatrix}(t, t')   
\end{split}\end{align}

#### Equal-Time.

\begin{align}\begin{split}
    0 &= \begin{pmatrix}
    i \partial_T + i(\lambda + \gamma) & 0 \\
    0 & i \partial_T
    \end{pmatrix} 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)
    - \left[\begin{pmatrix}
    \omega_1 & J \\
    J & \omega_2
    \end{pmatrix}, 
    \begin{pmatrix}
    G^<_{11} & G^<_{12} \\
    G^<_{21} & G^<_{22}
    \end{pmatrix}(T, 0)\right] 
    - i\gamma
    \begin{pmatrix}
    G^T_{11} + G^{\tilde{T}}_{11} & G^\tilde{T}_{12} \\
    G^{{T}}_{21} & 0
    \end{pmatrix}(T, 0)       \\
    0 &= \begin{pmatrix}
    i \partial_T - i(\lambda + \gamma) & 0 \\
    0 & i \partial_T
    \end{pmatrix} 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)  
    - \left[\begin{pmatrix}
    \omega_1 & J \\
    J & \omega_2
    \end{pmatrix}, 
    \begin{pmatrix}
    G^>_{11} & G^>_{12} \\
    G^>_{21} & G^>_{22}
    \end{pmatrix}(T, 0)\right]
    + i\lambda
    \begin{pmatrix}
    G^T_{11} + G^{\tilde{T}}_{11} & G^T_{12} \\
    G^{\tilde{T}}_{21} & 0
    \end{pmatrix}(T, 0)   
\end{split}\end{align}

## Solving

In [ ]:
# time parameters
T = 16.0

# Non-Hermitian Hamiltonian and jump operator
λ = 2.0
ω₁ = 2.5 * λ
ω₂ = 0.0
J = pi / 4 * λ

γ = λ

N₁ = 1.
N₂ = 0.1

H = ComplexF64[ω₁ - 0.5im * ((N₁ + 1) * λ + N₁ * γ) J; J ω₂ - 0.5im * ((N₂ + 1) * λ + N₂ * γ)]
# h = H;

In [ ]:
# quantum numbers
L = 2

# Allocate the initial Green functions (time arguments at the end)
GL = GreenFunction(zeros(ComplexF64, L, L, 1, 1), SkewHermitian)
GG = GreenFunction(zeros(ComplexF64, L, L, 1, 1), SkewHermitian)

# initial condition
n₀ = 2
GL[2, 2, 1, 1] = -1.0im * n₀
GG[2, 2, 1, 1] = -1.0im * (n₀ + 1)
GG[1, 1, 1, 1] = -1.0im;

In [ ]:
function fv!(out, times, t, t′)
    out[1] = -1.0im * (H * GL[t, t′] + [[1.0im * N₁ * γ, 0] [0, 1.0im * N₂ * γ]] * GL[t, t′])
    out[2] = -1.0im * (adjoint(H) * GG[t, t′] - 1.0im * λ * [[(N₁ + 1), 0] [0, (N₂ + 1)]] * GG[t, t′])
end

function fd!(out, times, t, t′)
    println("t: $(times[t])")
    out[1] = (-1.0im * (H * GL[t, t] - GL[t, t] * adjoint(H)
             + 1.0im * γ  * [[N₁ * (GL[1, 1, t, t] + GG[1, 1, t, t]), (N₁ + N₂) * (GL[2, 1, t, t] + GG[2, 1, t, t]) / 2] [(N₁ + N₂) * (GL[1, 2, t, t] + GG[1, 2, t, t]) / 2, N₂ * (GL[2, 2, t, t] + GG[2, 2, t, t])]])
             )
    out[2] = (-1.0im * (adjoint(H) * GG[t, t] - GG[t, t] * H
             - 1.0im * λ * [[(N₁ + 1) * (GL[1, 1, t, t] + GG[1, 1, t, t]), (N₁ + N₂ + 2) * (GG[2, 1, t, t] + GL[2, 1, t, t]) / 2] [(N₁ + N₂ + 2) * (GG[1, 2, t, t] + GL[1, 2, t, t]) / 2, (N₂ + 1) * (GL[2, 2, t, t] + GG[2, 2, t, t])]])
             )
end

In [ ]:
# call the solver
sol = kbsolve!(fv!, fd!, [GL, GG], (0.0, T); atol=1e-6, rtol=1e-4);

In [ ]:
sol.t |> size

### Fourier transform

In [ ]:
t_lin = range(first(sol.t), stop=last(sol.t), length=length(sol.t))
ρ_11_itp = interpolate((sol.t, sol.t), view(GL.data .- GG.data, 1, 1, :, : ), Gridded(Linear()));
ρ_22_itp = interpolate((sol.t, sol.t), view(GL.data .- GG.data, 2, 2, :, : ), Gridded(Linear()));

In [ ]:
ρ_11_wigner, (taus, ts) = wigner_transform([ρ_11_itp(t1, t2) for t1 in t_lin, t2 in t_lin]; ts=t_lin, fourier=false);
ρ_11_wigner = ρ_11_wigner |> imag;

ρ_22_wigner, (taus, ts) = wigner_transform([ρ_22_itp(t1, t2) for t1 in t_lin, t2 in t_lin]; ts=t_lin, fourier=false);
ρ_22_wigner = ρ_22_wigner |> imag;

In [ ]:
ρ_11_FFT, (ωs, ts) = wigner_transform([ρ_11_itp(t1, t2) for t1 in t_lin, t2 in t_lin]; ts=t_lin, fourier=true);
ρ_11_FFT = ρ_11_FFT |> imag;

ρ_22_FFT, (ωs, ts) = wigner_transform([ρ_22_itp(t1, t2) for t1 in t_lin, t2 in t_lin]; ts=t_lin, fourier=true);
ρ_22_FFT = ρ_22_FFT |> imag;

## Plotting

In [ ]:
function meshgrid(xin, yin)
  nx=length(xin)
  ny=length(yin)
  xout=zeros(ny, nx)
  yout=zeros(ny, nx)
  for jx=1:nx
      for ix=1:ny
          xout[ix, jx]=xin[jx]
          yout[ix, jx]=yin[ix]
      end
  end
  return (x=xout, y=yout)
end

In [ ]:
t_scale = λ
ω_scale = 1/λ
steps = 1
cmap = "gist_heat";

In [ ]:
Y, X = meshgrid(sol.t[1:steps:end], sol.t[1:steps:end]);

In [ ]:
xpad = 8
ypad = 5

figure(figsize=(7, 3))

ax = subplot(121)

plot(t_scale * sol.t, [-imag(GL.data[1, 1, k, k]) for k in 1:length(sol.t)], ls="--", c="C3", label=L"i=1", lw=1.5)
plot(t_scale * sol.t, [-imag(GL.data[2, 2, k, k]) for k in 1:length(sol.t)], ls="-", c="C0", label=L"i=2", lw=1.5)

ax.set_xlim(0, t_scale * T/2)
ax.set_xticks(t_scale .* [0, T/4, T/2])
ax.set_ylim(0, n₀)
ax.set_xlabel(L"\lambda T")
ax.set_ylabel(L"-\mathrm{Im}\; G^<_{ii}(t, t)")
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.legend(loc="best", handlelength=1.9, frameon=false, borderpad=0, labelspacing=0.25)

ax = subplot(122)
X, Y = meshgrid(ts[1:steps:end], taus[1:steps:end]);
vmin = 1.0
vmax = -0.5

heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, ρ_11_wigner[1:steps:end, 1:steps:end], cmap=cmap, vmin=vmin, vmax=vmax, rasterized=true)
heatmap.set_edgecolor("face")

ax.set_xlabel(L"\lambda T")
ax.set_ylabel(L"\lambda \tau")
ax.set_xlim(0, t_scale * T)
ax.set_xticks([0, t_scale * T/2, t_scale * T])
ax.set_ylim(-t_scale * T/2, t_scale * T/2)
ax.set_yticks([-t_scale * T/2, 0, t_scale * T/2])
colorbar(mappable=heatmap)
ax.set_aspect("equal")
tight_layout()
# savefig("boson_example_1.pdf")

In [ ]:
xpad = 8
ypad = 5

figure(figsize=(7, 3))

ax = subplot(121)
plot(t_scale * taus, ρ_11_wigner[:, Int(floor(length(taus)/2))], ls="--", c="C3", label=L"i=1", lw=1.5) # fixed T
plot(t_scale * taus, ρ_22_wigner[:, Int(floor(length(taus)/2))], ls="-", c="C0", label=L"i=2", lw=1.5)
ax.set_xlabel(L"\lambda \tau")
ax.set_xlim(-t_scale * T/2, t_scale * T/2)
ax.set_ylim(-0.5, 1.0)
ax.set_xticks(t_scale .* [-T/2, -T/4, 0, T/4, T/2])
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_ylabel(L"\textrm{Re}\, A_{ii}(T, \tau)_W")
ax.legend(loc="best", handlelength=1.4, frameon=false, borderpad=0, labelspacing=0.25)

ax = subplot(122)
plot(ω_scale * ωs, ρ_11_FFT[:, Int(floor(length(taus)/2))], ls="--", c="C3", lw=1.5)
plot(ω_scale * ωs, ρ_22_FFT[:, Int(floor(length(taus)/2))], ls="-", c="C0", lw=1.5)
ax.set_xlabel(L"\omega/\lambda")
ax.set_xlim(10 .* (-1, 1))
ax.set_xticks([-10, -5, 0, 5, 10])
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_ylabel(L"\textrm{Re}\, A_{ii}(T, \omega)_{\tilde{W}}", labelpad=16)
ax.yaxis.set_label_position("right")
ax.yaxis.set_ticks_position("both")
tight_layout(pad=0.1, w_pad=0.5, h_pad=0)

# savefig("boson_example_2.pdf")